**Here we are going to clean the data for manipulating the dataset later by saving/loading a dataframe**

Finish with thi sline of code:

In [142]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

**Dataframe Preparation**

In [143]:
food = pd.read_csv('data/food-inspections.csv', sep=',')

In [144]:
food = food.drop(['DBA Name','Address','City', 'State','Zip', 'Zip Codes','Historical Wards 2003-2015', 'Community Areas', 'Census Tracts','Wards','Location'], axis=1)

In [145]:
food.head()

,Inspection ID,AKA Name,License #,Facility Type,Risk,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
0,2320830,"THE HOXTON, CHICAGO",2694640.0,Restaurant,Risk 2 (Medium),2019-10-31T00:00:00.000,License,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.885699,-87.648789
1,2320831,OGDEN PLAZA INC.,2475982.0,Grocery Store,Risk 3 (Low),2019-10-31T00:00:00.000,Canvass,Out of Business,NaN,41.855266,-87.712402
2,2320829,PLAZA FOOD AND LIQUOR,2689756.0,Grocery Store,Risk 3 (Low),2019-10-31T00:00:00.000,License,Not Ready,NaN,NaN,NaN
3,2320813,PLAZA FOOD AND LIQUOR,2689757.0,Grocery Store,Risk 3 (Low),2019-10-31T00:00:00.000,License,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,NaN,NaN
4,2320757,GADS HILL CENTER,2698627.0,Daycare Above and Under 2 Years,Risk 1 (High),2019-10-30T00:00:00.000,License,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.816005,-87.700893


**Cleaning steps**

1) Drop rows without location.

2) Filter only the 4 facility types we are going to analyse: restaurants, grocery stores, schools and hospitals.

3) Convert the date format to an analysis friendly format

4) There are 3 risk levels (low, medium and high), any other value will be removed

5) For restaurants, we are going to focus only the 5 most inspected chains of different types: McDonald's, Subway, Taco Bell, Satrbucks and Dunkin Donuts.

**Step 1**

In [146]:
food.Latitude.isna().any() or food.Longitude.isna().any()

True

In [147]:
food.dropna(subset = ["Latitude", "Longitude"], inplace=True)
food.Latitude.isna().any() or food.Longitude.isna().any()

False

**Step 2**

In [148]:
food = food[food["Facility Type"].isin(["Restaurant","Grocery Store", "School", "Hospital"])]
food["Facility Type"].value_counts()

Restaurant       129891
Grocery Store     24838
School            11808
Hospital            537
Name: Facility Type, dtype: int64

**Step 3**

In [149]:
food["Inspection Date"] = food["Inspection Date"].str.split("-").str[0]
food.head()

,Inspection ID,AKA Name,License #,Facility Type,Risk,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
0,2320830,"THE HOXTON, CHICAGO",2694640.0,Restaurant,Risk 2 (Medium),2019,License,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.885699,-87.648789
1,2320831,OGDEN PLAZA INC.,2475982.0,Grocery Store,Risk 3 (Low),2019,Canvass,Out of Business,NaN,41.855266,-87.712402
6,2320795,THE EXCHANGE,2698572.0,Restaurant,Risk 1 (High),2019,License,Pass,NaN,41.887529,-87.632647
7,2320768,The Manor,22971.0,Restaurant,Risk 1 (High),2019,Canvass,No Entry,NaN,41.807924,-87.728164
9,2320719,"4884 S ARCHER INC,.",2678088.0,Grocery Store,Risk 3 (Low),2019,License,Fail,NaN,41.804621,-87.719907


In [150]:
#Check for missing years
food["Inspection Date"].isna().any()

False

**Step 4**

In [151]:
#checking for NaN's
food.dropna(subset = ["Risk"], inplace = True)
food.Risk.isna().any()

False

In [152]:
#Checking for any other value than the 3 levels of risk
food.Risk.value_counts()

Risk 1 (High)      123056
Risk 2 (Medium)     33625
Risk 3 (Low)        10378
All                    10
Name: Risk, dtype: int64

In [153]:
#cleaning
food = food[~food["Risk"].str.contains('|'.join(["All"]))]
food.Risk.value_counts()

Risk 1 (High)      123056
Risk 2 (Medium)     33625
Risk 3 (Low)        10378
Name: Risk, dtype: int64

**Step 5**

In [154]:
#Preview of restaurants
restaurants = food[food["Facility Type"]=="Restaurant"].copy()
restaurants["AKA Name"].value_counts().head(25)

SUBWAY                          3203
DUNKIN DONUTS                   1327
MCDONALD'S                       758
BURGER KING                      372
MCDONALDS                        335
CHIPOTLE MEXICAN GRILL           319
DUNKIN DONUTS/BASKIN ROBBINS     319
WENDY'S                          296
STARBUCKS COFFEE                 288
POTBELLY SANDWICH WORKS          271
CORNER BAKERY CAFE               240
FRESHII                          237
STARBUCKS                        235
JIMMY JOHN'S                     223
Subway                           223
PIZZA HUT                        213
SUBWAY SANDWICHES                206
DOMINO'S PIZZA                   201
TACO BELL                        200
KFC                              197
AU BON PAIN                      185
POTBELLY SANDWICH WORKS LLC      177
HAROLD'S CHICKEN SHACK           170
MC DONALD'S                      169
SEE THRU CHINESE KITCHEN         169
Name: AKA Name, dtype: int64

In [155]:
#Drop NaN's
restaurants.dropna(subset=["AKA Name"], inplace=True)
restaurants["AKA Name"].isna().any()

False

In [156]:
#Only uppercase
restaurants["AKA Name"] = restaurants["AKA Name"].str.upper()

Now that a first cleaning was done, we will concentrate on the 5 restaurants chosen and mentioned before

**McDonald's**

In [157]:
#Let's see if there is other restaurants with "donald" to avoid adding wrong data to McDonald's
restaurants[restaurants["AKA Name"].str.contains("DONALD")]["AKA Name"].value_counts()

MCDONALD'S                           906
MCDONALDS                            404
MC DONALD'S                          169
MC DONALDS                           168
MCDONALD'S RESTAURANT                 87
MCDONALDS RESTAURANT                  43
MCDONALD'S RESTAURANTS                24
MCDONALD'S #490                       20
DONALDS FAMOUS HOT DOGS               19
MCDONALD'S RESTAURANT  (T3 H9)        18
MCDONALD'S  (T3 HK FOOD COURT)        17
MCDONALD'S CORPORATION                17
MC DONALDS # 6771                     16
MCDONALDS #27672                      15
MCDONALDS  (T3  K9)                   15
MCDONALD' S # 5618                    15
MCDONALDS #4655                       14
MCDONALD'S   (T3- L4)                 13
MCDONALDS #7069                       13
MCDONALDS#6337                        12
MCDONALD'S #20104                     11
MCDONALD'S  (T2   E/F)                11
MCDONALD'S  (T1-B11)                  11
MCDONALD'S STORE #4061                11
MC DONALDS-MCCOR

In [158]:
#removing "Donald's famous hot dogs" than unifying all McDonald's
restaurants = restaurants[~restaurants["AKA Name"].str.contains('|'.join(["DOGS"]))]
restaurants.loc[restaurants["AKA Name"].str.contains("DONALD"), "AKA Name"] = "MCDONALDS"
restaurants[restaurants["AKA Name"].str.contains("DONALD")]["AKA Name"].value_counts()

MCDONALDS    2218
Name: AKA Name, dtype: int64

**Subway**

In [159]:
#Let's see if there is other restaurants with "subway" to avoid adding wrong data to Subway
restaurants[restaurants["AKA Name"].str.contains("SUBWAY")]["AKA Name"].value_counts()

SUBWAY                                           3426
SUBWAY SANDWICHES                                 232
SUBWAY SANDWICH                                    60
SUBWAY SANDWICH & SALAD                            30
SUBWAY (T3 ROTUNDA)                                25
SUBWAY RESTAURANT                                  25
SUBWAY #3333                                       21
SUBWAY SANDWICHES & SALADS                         21
SUBWAY 28330                                       18
SNAPPY CONVENIENCE CENTER/SUBWAY/DUNKIN DONUT      18
BP/SUBWAY                                          17
SUBWAY #45927                                      16
SUBWAY SANDWICH STORE                              16
SUBWAY 48735                                       14
SHELL SUBWAY                                       12
ROAD RANGER/SUBWAY                                 11
LALO SUBWAY INC                                    10
LAKEVIEW SUBWAY                                     9
MADISON SUBWAY LLC          

In [160]:
#removing restaurants with "subway" undesired than unifying all Subway's
removable = ["FULLERTON","MADISON", "LALO","LAKEVIEW","SNAPPY"]
restaurants = restaurants[~restaurants["AKA Name"].str.contains('|'.join(removable))]
restaurants.loc[restaurants["AKA Name"].str.contains("SUBWAY"), "AKA Name"] = "SUBWAY"
restaurants[restaurants["AKA Name"].str.contains("SUBWAY")]["AKA Name"].value_counts()

SUBWAY    3988
Name: AKA Name, dtype: int64

**Starbucks**

In [161]:
restaurants[restaurants["AKA Name"].str.contains("STARBUCKS")]["AKA Name"].value_counts().head(40)

STARBUCKS COFFEE                                   301
STARBUCKS                                          235
MARKET PLACE/STARBUCKS COFFE/FRANGO/GODIVA          22
STARBUCKS COFFEE #2370                              14
STARBUCKS COFFEE #2334                              13
STARBUCKS HK APEX (T3 HK FOODCOURT)                 13
STARBUCKS COFFEE (T1-B5)                            13
STARBUCKS (T1/B CONCOURSE-BAGGAGE CLAIM)            12
STARBUCKS/ W KITCHEN/ F1,F2 POD/WAREHSE/LA BREA     12
STARBUCKS (T2 LL ARRIVAL)                           12
STARBUCKS (T3  G14 LL)                              12
STARBUCKS COFFEE #2410                              12
MAIN KITCHEN/STARBUCKS /ETA/ EMPL CAFE              11
STARBUCKS COFFEE #228                               11
STARBUCKS COFFEE  #2635                             11
STARBUCKS COFFEE #2527                              11
STARBUCKS COFFEE #9942                              11
STARBUCKS  (T3 H6)                                  11
STARBUCKS 

In [162]:
#we are going to assume "starbucks" is a very distinctive name and have negligible chance of being used in another restaurant
restaurants.loc[restaurants["AKA Name"].str.contains("STARBUCKS"), "AKA Name"] = "STARBUCKS"
restaurants[restaurants["AKA Name"].str.contains("STARBUCKS")]["AKA Name"].value_counts()

STARBUCKS    1451
Name: AKA Name, dtype: int64

**Taco Bell**

In [163]:
#Checking a misspelled case
restaurants[restaurants["AKA Name"].str.contains("TACOBELL")]["AKA Name"].value_counts()

Series([], Name: AKA Name, dtype: int64)

In [164]:
#Looks like Taco Bell is always written in separated words
restaurants[restaurants["AKA Name"].str.contains("TACO BELL")]["AKA Name"].value_counts()

TACO BELL                         206
KFC/TACO BELL                      25
TACO BELL #15855                   15
TACO BELL & LONG JOHN SILVER'S     13
TACO BELL #30407                   11
TACO BELL CANTINA                  11
TACO BELL #2513                     9
TACO BELL_#4171                     9
TACO BELL #15875                    9
TACO BELL #5751                     8
TACO BELL 32575                     3
TACO BELL 34921                     2
Name: AKA Name, dtype: int64

In [165]:
#And the name is luckly very unique
restaurants.loc[restaurants["AKA Name"].str.contains("TACO BELL"), "AKA Name"] = "TACO BELL"
restaurants[restaurants["AKA Name"].str.contains("TACO BELL")]["AKA Name"].value_counts()

TACO BELL    321
Name: AKA Name, dtype: int64

**Dunkin Donuts**

In [166]:
#Checking a misspelled case
restaurants[restaurants["AKA Name"].str.contains("DUNKINDONUTS")]["AKA Name"].value_counts()

DUNKINDONUTS    7
Name: AKA Name, dtype: int64

When unifying all Dunkin Donuts we will also need to add the 7 "DUNKINDONUTS" from above

In [167]:
restaurants[restaurants["AKA Name"].str.contains("DUNKIN DONUTS")]["AKA Name"].value_counts()

DUNKIN DONUTS                           1380
DUNKIN DONUTS/BASKIN ROBBINS             319
DUNKIN DONUTS BASKIN ROBBINS             122
DUNKIN DONUTS / BASKIN ROBBINS           117
DUNKIN DONUTS/ BASKIN ROBBINS             60
DUNKIN DONUTS / BASKIN ROBINS             47
DUNKIN DONUTS & BASKIN ROBBINS            28
DUNKIN DONUTS / BASKIN & ROBBINS          25
DUNKIN DONUTS & BASKIN ROBINS             15
HALSTED SHELL, DUNKIN DONUTS, MR SUB      14
DUNKIN DONUTS-BASKIN ROBBINS              14
BASKIN ROBBINS/ DUNKIN DONUTS             13
DUNKIN DONUTS BASKIN ROBBINS TOGO'S       12
DUNKIN DONUTS (T3 HK FOODCOURT)           11
GRAND CITGO/ MR. SUB / DUNKIN DONUTS      10
BASKIN ROBBINS/DUNKIN DONUTS              10
DUNKIN DONUTS/BASKIN ROBINS               10
DUNKIN DONUTS /  BASKIN ROBBINS           10
DUNKIN DONUTS INC                          9
DUNKIN DONUTS/BASKIN  ROBBINS              8
DUNKIN DONUTS AND BASKIN ROBBINS           4
DUNKIN DONUTS /  BASKIN ROBINS             4
JETBOX WIT

In [168]:
#Dunkin Donuts is very unique too which helps us when unifying
restaurants.loc[restaurants["AKA Name"].str.contains("DUNKIN DONUTS"), "AKA Name"] = "DUNKIN DONUTS"
restaurants.loc[restaurants["AKA Name"].str.contains("DUNKINDONUTS"), "AKA Name"] = "DUNKIN DONUTS"
restaurants[restaurants["AKA Name"].str.contains("DUNKIN DONUTS")]["AKA Name"].value_counts()

DUNKIN DONUTS    2258
Name: AKA Name, dtype: int64

**Removing any other restaurants**

In [169]:
restaurant_list = ["MCDONALDS","SUBWAY","STARBUCKS","TACO BELL","DUNKIN DONUTS"]
restaurants = restaurants[restaurants["AKA Name"].isin(restaurant_list)]
restaurants["AKA Name"].value_counts()

SUBWAY           3988
DUNKIN DONUTS    2258
MCDONALDS        2218
STARBUCKS        1451
TACO BELL         321
Name: AKA Name, dtype: int64

**Final Step**

In [170]:
#Reset Indexes after cleaning
food.reset_index(drop=True, inplace=True)
restaurants.reset_index(drop=True, inplace=True)

In [171]:
#save pickles for use in analysis
food.to_pickle("food.pkl")
restaurants.to_pickle("restaurants.pkl")